In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [2]:
df = pd.read_csv('../new_featured_dataset.csv')

In [3]:
df.head()

,Mean,std,Entropy,RMS,Kurtios,HMI1,HMI2,HMI3,HMI4,Area,...,homogeneity4,energy1,energy2,energy3,energy4,correlation1,correlation2,correlation3,correlation4,Class
0,55.212544,46.056565,5.375762,9.302612,-0.628678,2.752905,8.030957,11.070417,11.550219,10522.700000,...,0.513325,0.177499,0.172328,0.180325,0.171641,0.969317,0.959636,0.974818,0.958198,0
1,76.037273,79.565990,5.781641,7.635726,-0.877843,2.904895,7.803627,12.448359,12.308773,4735.000000,...,0.432991,0.230150,0.221221,0.232777,0.221043,0.969677,0.958555,0.973180,0.957760,0
2,53.652053,59.587603,4.946379,10.302398,2.218936,2.779911,7.267831,10.447366,12.249549,10367.875000,...,0.490227,0.232071,0.220649,0.233649,0.221646,0.949499,0.932350,0.958399,0.931763,0
3,49.231160,42.645217,6.123446,10.533521,-1.392569,2.778616,7.928608,10.892414,12.092358,9536.166667,...,0.450451,0.374133,0.365637,0.379210,0.367641,0.913667,0.891387,0.927097,0.891294,0
4,77.261385,83.705507,6.017668,7.935202,-0.774676,2.996775,7.602490,11.633094,12.014271,18001.000000,...,0.444437,0.312793,0.304938,0.317934,0.302303,0.923784,0.898480,0.934894,0.898350,0


In [4]:
# df = df.dropna()

missing_data = ['Area', 'Perimeter', 'Circle', 'Convex Area', 'Solidity', 'Equivalent Diameter', 'Major Axis', 'Minor Axis']
for i in missing_data:
    fill_list = df[i].dropna().tolist()
    # df[i] = df[i].fillna(pd.Series(np.random.choice(fill_list , size = len(df.index))))
    mean_fill = np.mean(fill_list)
    df[i] = df[i].fillna(mean_fill)
df.isnull().sum()

Mean                   0
std                    0
Entropy                0
RMS                    0
Kurtios                0
HMI1                   0
HMI2                   0
HMI3                   0
HMI4                   0
Area                   0
Perimeter              0
Circle                 0
Convex Area            0
Solidity               0
Equivalent Diameter    0
Major Axis             0
Minor Axis             0
contrast1              0
contrast2              0
contrast3              0
contrast4              0
dissimilarity1         0
dissimilarity2         0
dissimilarity3         0
dissimilarity4         0
homogeneity1           0
homogeneity2           0
homogeneity3           0
homogeneity4           0
energy1                0
energy2                0
energy3                0
energy4                0
correlation1           0
correlation2           0
correlation3           0
correlation4           0
Class                  0
dtype: int64

In [5]:
X = df.drop(columns=["Class"])
y = df["Class"]

## feature selection

In [6]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Initialize the feature selector
k_best = SelectKBest(score_func=mutual_info_classif, k=20)  # Select the top 2 features based on mutual information

feature_names = ['Mean', 'std', 'Entropy', 'RMS', 'Kurtios', 'HMI1', 'HMI2', 'HMI3',
       'HMI4', 'Area', 'Perimeter', 'Circle', 'Convex Area', 'Solidity',
       'Equivalent Diameter', 'Major Axis', 'Minor Axis', 'contrast1',
       'contrast2', 'contrast3', 'contrast4', 'dissimilarity1',
       'dissimilarity2', 'dissimilarity3', 'dissimilarity4', 'homogeneity1',
       'homogeneity2', 'homogeneity3', 'homogeneity4', 'energy1', 'energy2',
       'energy3', 'energy4', 'correlation1', 'correlation2', 'correlation3',
       'correlation4']

# Apply feature selection to the data
X_selected = k_best.fit_transform(X, y)

# Get the selected feature indices and scores
selected_indices = k_best.get_support(indices=True)
selected_scores = k_best.scores_

# Sort the features based on scores in ascending order
sorted_features = sorted(zip(selected_indices, selected_scores), key=lambda x: x[1], reverse=True)
X_select = pd.DataFrame()
# Print the top 20 features in ascending order
ll=[]
for idx, score in sorted_features[:20]:
    ll.append(feature_names[idx])
    print(f"Feature {feature_names[idx]}: Score {score}")

Feature Mean: Score 0.47114247635587847
Feature std: Score 0.4704960449659905
Feature contrast2: Score 0.4661881218395143
Feature correlation2: Score 0.46088780747691893
Feature correlation3: Score 0.4503137386038183
Feature correlation4: Score 0.4443888408258654
Feature HMI1: Score 0.30537128586209317
Feature contrast1: Score 0.2910232825428394
Feature RMS: Score 0.2804242973388842
Feature energy1: Score 0.2764622531792984
Feature energy2: Score 0.26668708119079554
Feature dissimilarity2: Score 0.2484815920396004
Feature dissimilarity4: Score 0.2314353008060248
Feature correlation1: Score 0.23100753213979797
Feature energy3: Score 0.2298266644235456
Feature contrast3: Score 0.22858090213767568
Feature energy4: Score 0.19928852514920647
Feature dissimilarity1: Score 0.15566827057320065
Feature contrast4: Score 0.1389583480224612
Feature dissimilarity3: Score 0.12807648503937052


## select best features only

In [7]:
X_select = X[ll]

In [8]:
X_select

,Mean,std,contrast2,correlation2,correlation3,correlation4,HMI1,contrast1,RMS,energy1,energy2,dissimilarity2,dissimilarity4,correlation1,energy3,contrast3,energy4,dissimilarity1,contrast4,dissimilarity3
0,55.212544,46.056565,183.902674,0.959636,0.974818,0.958198,2.752905,140.013325,9.302612,0.177499,0.172328,4.735315,4.859588,0.969317,0.180325,114.986416,0.171641,3.831805,190.455883,3.082982
1,76.037273,79.565990,502.955136,0.958555,0.973180,0.957760,2.904895,368.273419,7.635726,0.230150,0.221221,8.761257,8.872275,0.969677,0.232777,325.904539,0.221043,6.768184,512.597992,6.198473
2,53.652053,59.587603,551.439410,0.932350,0.958399,0.931763,2.779911,411.974803,10.302398,0.232071,0.220649,8.844310,8.784037,0.949499,0.233649,339.474792,0.221646,6.668132,556.220494,6.031714
3,49.231160,42.645217,535.392752,0.891387,0.927097,0.891294,2.778616,427.392473,10.533521,0.374133,0.365637,10.431541,10.319736,0.913667,0.379210,361.157438,0.367641,8.527744,535.851169,7.879728
4,77.261385,83.705507,996.340638,0.898480,0.934894,0.898350,2.996775,753.461519,7.935202,0.312793,0.304938,14.913752,14.967293,0.923784,0.317934,644.759621,0.302303,11.950924,997.611651,10.607111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5212,55.537216,41.476325,360.145406,0.914916,0.949625,0.912128,2.678192,285.485452,9.934108,0.149783,0.145493,7.329037,7.500077,0.932639,0.154335,213.722145,0.144780,5.922500,371.946740,4.884709
5213,58.451622,42.845989,325.038720,0.921140,0.955264,0.920016,2.700193,263.393509,10.103457,0.136611,0.132942,7.204017,7.338640,0.936218,0.141475,184.926555,0.131985,5.841866,329.672273,4.831209
5214,45.893677,33.173744,208.513996,0.915585,0.950032,0.910542,2.585893,164.600526,9.474798,0.081568,0.077379,5.633784,5.813055,0.933455,0.082941,123.585835,0.076869,4.430714,220.971159,3.914038
5215,41.703392,36.594254,260.407608,0.926110,0.953305,0.926715,2.608306,185.530025,9.738114,0.100671,0.096011,6.705807,6.656449,0.947402,0.102649,164.764602,0.095976,5.071237,258.275182,4.619081


# models with whole dataset

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## RF

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
# Fit the parameters before oversampling:
rf_clf.fit(X_train, y_train)
print(f"Train Acc of Random Forest : {rf_clf.score(X_train,y_train)}")
# rf_clf.score(X_test,y_test)
print(f"Test Acc of Random Forest : {rf_clf.score(X_test,y_test)}")

Train Acc of Random Forest : 1.0
Test Acc of Random Forest : 0.8850574712643678


## SVM

In [11]:
from sklearn.svm import SVC
clf1 = SVC(kernel='rbf',C=100,gamma=0.001)
clf1.fit(X_train, y_train)
print(f"Train Acc of SVM : {clf1.score(X_train,y_train)}")
print(f"Test Acc of SVM : {clf1.score(X_test,y_test)}")

Train Acc of SVM : 1.0
Test Acc of SVM : 0.5153256704980843


## KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier

# create KNeighborsClassifier with 7 neighbors 
knn = KNeighborsClassifier(n_neighbors=7)
# Fit the data to knn model
knn.fit(X_train,y_train)
print(f"Train acc of KNN: {knn.score(X_train,y_train)}")
print(f"Test acc of KNN: {knn.score(X_test,y_test)}")

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train acc of KNN: 0.7335250419362569
Test acc of KNN: 0.6628352490421456


c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## PCA and StandardScaler

In [13]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca_m = PCA(n_components=37)

new_X = pca_m.fit_transform(X)
scaler = StandardScaler()
new_X = scaler.fit_transform(new_X)

## split data after pca and scaler

In [14]:
X_trainn,X_testn,y_trainn,y_testn = train_test_split(new_X,y,test_size=0.2,random_state=42)

## RF PCA

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf_clf2 = RandomForestClassifier()
# Fit the parameters before oversampling:
rf_clf2.fit(X_trainn, y_trainn)
print(f"Train Acc of Random Forest PCA : {rf_clf2.score(X_trainn,y_trainn)}")
print(f"Test Acc of Random Forest PCA : {rf_clf2.score(X_testn,y_testn)}")

Train Acc of Random Forest PCA : 1.0
Test Acc of Random Forest PCA : 0.8955938697318008


## SVM PCA

In [16]:
from sklearn.svm import SVC
clf2 = SVC(kernel='linear',C=100,gamma=0.001)
clf2.fit(X_trainn, y_trainn)

print(f"Train Acc of SVM PCA : {clf2.score(X_trainn,y_trainn)}")
print(f"Test Acc of SVM PCA : {clf2.score(X_testn,y_testn)}")

Train Acc of SVM PCA : 0.8698777857656362
Test Acc of SVM PCA : 0.8496168582375478


## KNN PCA

In [17]:
knn2 = KNeighborsClassifier(n_neighbors=7)
# Fit the data to knn model
knn2.fit(X_trainn,y_trainn)
print(f"Train acc of KNN PCA : {knn2.score(X_trainn,y_trainn)}")
print(f"Test acc of KNN PCA : {knn2.score(X_testn,y_testn)}")

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train acc of KNN PCA : 0.9084591421040019
Test acc of KNN PCA : 0.8505747126436781


c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


# selected features

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_select,y,test_size=0.2,random_state=42)

## RF Selected

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
# Fit the parameters before oversampling:
rf_clf.fit(X_train, y_train)
print(f"Train Acc of Random Forest : {rf_clf.score(X_train,y_train)}")
# rf_clf.score(X_test,y_test)
print(f"Test Acc of Random Forest : {rf_clf.score(X_test,y_test)}")

Train Acc of Random Forest : 1.0
Test Acc of Random Forest : 0.8754789272030651


## SVM Selected

In [20]:
from sklearn.svm import SVC
clf1 = SVC(kernel='rbf',C=100,gamma=0.001)
clf1.fit(X_train, y_train)
print(f"Train Acc of SVM : {clf1.score(X_train,y_train)}")
print(f"Test Acc of SVM : {clf1.score(X_test,y_test)}")

Train Acc of SVM : 0.9453630481667865
Test Acc of SVM : 0.7701149425287356


## KNN Selected

In [21]:
from sklearn.neighbors import KNeighborsClassifier

# create KNeighborsClassifier with 7 neighbors 
knn = KNeighborsClassifier(n_neighbors=7)
# Fit the data to knn model
knn.fit(X_train,y_train)
print(f"Train acc of KNN: {knn.score(X_train,y_train)}")
print(f"Test acc of KNN: {knn.score(X_test,y_test)}")

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train acc of KNN: 0.7977474239156482
Test acc of KNN: 0.7413793103448276


c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## With PCA and StandardScaler

In [22]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca_m = PCA(n_components=20)

new_X = pca_m.fit_transform(X_select)
scaler = StandardScaler()
new_X = scaler.fit_transform(new_X)

## Split Selected

In [23]:
X_trainn,X_testn,y_trainn,y_testn = train_test_split(new_X,y,test_size=0.2,random_state=42)

## RF Selected PCA

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf_clf2 = RandomForestClassifier()
# Fit the parameters before oversampling:
rf_clf2.fit(X_trainn, y_trainn)
print(f"Train Acc of Random Forest PCA : {rf_clf2.score(X_trainn,y_trainn)}")
print(f"Test Acc of Random Forest PCA : {rf_clf2.score(X_testn,y_testn)}")

Train Acc of Random Forest PCA : 1.0
Test Acc of Random Forest PCA : 0.8917624521072797


## SVM Selected PCA

In [25]:
from sklearn.svm import SVC
clf2 = SVC(kernel='linear',C=100,gamma=0.001)
clf2.fit(X_trainn, y_trainn)

print(f"Train Acc of SVM PCA : {clf2.score(X_trainn,y_trainn)}")
print(f"Test Acc of SVM PCA : {clf2.score(X_testn,y_testn)}")

Train Acc of SVM PCA : 0.8070932183081716
Test Acc of SVM PCA : 0.8065134099616859


## KNN Selected PCA

In [26]:
knn2 = KNeighborsClassifier(n_neighbors=7)
# Fit the data to knn model
knn2.fit(X_trainn,y_trainn)
print(f"Train acc of KNN PCA : {knn2.score(X_trainn,y_trainn)}")
print(f"Test acc of KNN PCA : {knn2.score(X_testn,y_testn)}")

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train acc of KNN PCA : 0.8818595734483585
Test acc of KNN PCA : 0.8448275862068966


c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
